## Para este ejercicio uso la API de Spotify y extraigo los new releases de albums en Argentina y Uruguay para ampliar la cantidad de registros y también poder realizar algunas transformaciones de datos. La idea es generar un dataset con los 50 nuevos lanzamientos de Argentina y Uruguay, tener informacion del Artista, Album, Fecha de lanzamiento, canciones del album y cantidad de veces que fueron reproducidas cada una. Con esta información puedo saber por ejemplo que albumes son los mas escuchados y los menos escuchados para tomar alguna acción de marketing sobre eso.

Para ello uso la api de Spotify https://spotipy.readthedocs.io/en/2.22.1/


In [1]:
!pip install spotipy --upgrade
!pip install -U datasets
!pip install psycopg2-binary

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import os
import psycopg2

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col

In [18]:
env = os.environ

Creo un cliente para conectar a Spotify

In [19]:
# Configurar las credenciales de cliente de Spotify
client_id = env['SPOTIFY_CLIENT_ID']
client_secret = env['SPOTIFY_CLIENT_SECRET']

# Crear una instancia de SpotifyClientCredentials con las credenciales
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# Crear una instancia de la clase Spotipy
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Recupero los datos que necesito de Spotify y creo una lista para guardarlos

In [30]:
# Obtener los últimos lanzamientos en Argentina
results_ar = sp.new_releases(country='AR', limit=50)
albums_ar = results_ar['albums']['items']

# Obtener los últimos lanzamientos en Uruguay
results_uy = sp.new_releases(country='UY', limit=50)
albums_uy = results_uy['albums']['items']

# Combinar los resultados de Argentina y Uruguay
all_albums = albums_ar + albums_uy

# Crear una lista para almacenar los datos
data = []

for album in all_albums:
    artist_name = album['artists'][0]['name']
    album_name = album['name']
    release_date = album['release_date']
    album_id = album['id']
     
    # Obtener las canciones del álbum
    tracks = sp.album_tracks(album_id)
    
    for i, track in enumerate(tracks['items']):
        track_name = track['name']
        
        # Obtener las estadísticas de reproducción de la canción
        track_stats = sp.track(track['id'])
        num_playbacks = track_stats['popularity']
        
        data.append((artist_name , album_name , release_date, track_name, num_playbacks))

data

[('Bizarrap',
  'Peso Pluma: Bzrp Music Sessions, Vol. 55',
  '2023-06-01',
  'Peso Pluma: Bzrp Music Sessions, Vol. 55',
  93),
 ('LIT killah', 'Los del Espacio', '2023-06-01', 'Los del Espacio', 85),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'El Amor Después del Amor - EADDA9223',
  59),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'Dos Días en la Vida - EADDA9223 (feat. Lali & Nicki Nicole)',
  64),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'La Verónica - EADDA9223 (feat. NATHY PELUSO)',
  58),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'Tráfico por Katmandú - EADDA9223 (feat. Elvis Costello)',
  57),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'Pétalo de Sal - EADDA9223 (feat. Chico Buarque)',
  57),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'Sasha, Sissi y el Círculo de Baba - EADDA9223 (feat. Mon Laferte)',
  59),
 ('Fito Paez',
  'EADDA9223',
  '2023-05-30',
  'Un Vestido y un Amor - EADDA9223 (feat. Marisa Monte)',
  57),
 ('Fito Paez',
  'EADDA922

In [161]:
# Postgres and Redshift JDBCs
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

Creación de un dataframe en spark con los datos obtenidos

In [163]:
# Creacion del DataFrame con los datos 
df = spark.createDataFrame(data, ['Artista','Album','Fecha_release','Track','Reprod_track'])
df.printSchema()
df.show()

root
 |-- Artista: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Fecha_release: string (nullable = true)
 |-- Track: string (nullable = true)
 |-- Reprod_track: long (nullable = true)

+----------+--------------------+-------------+--------------------+------------+
|   Artista|               Album|Fecha_release|               Track|Reprod_track|
+----------+--------------------+-------------+--------------------+------------+
|  Bizarrap|Peso Pluma: Bzrp ...|   2023-06-01|Peso Pluma: Bzrp ...|          93|
|LIT killah|     Los del Espacio|   2023-06-01|     Los del Espacio|          85|
| Fito Paez|           EADDA9223|   2023-05-30|El Amor Después d...|          59|
| Fito Paez|           EADDA9223|   2023-05-30|Dos Días en la Vi...|          64|
| Fito Paez|           EADDA9223|   2023-05-30|La Verónica - EAD...|          58|
| Fito Paez|           EADDA9223|   2023-05-30|Tráfico por Katma...|          57|
| Fito Paez|           EADDA9223|   2023-05-30|Pétalo de

ETAPA DE TRANSFORMACION - Suma de cantidad de reproducciones por canción, eliminacion de nulls, eliminacion de duplicados, transformacion de fecha tipo string a fecha tipo date, suma de cantidad de reproducciones en total del album (contando las reproducciones de cada cancion)

In [165]:
# eliminacion de nulls en cualquier columna
print ('cant registros dataframe',df_final.count())
df_final=df.na.drop(how="any")
print ('cant registros dataframe sin nulls',df_final.count())
df_final.show()

cant registros dataframe 852
cant registros dataframe sin nulls 852
+----------+--------------------+-------------+--------------------+------------+
|   Artista|               Album|Fecha_release|               Track|Reprod_track|
+----------+--------------------+-------------+--------------------+------------+
|  Bizarrap|Peso Pluma: Bzrp ...|   2023-06-01|Peso Pluma: Bzrp ...|          93|
|LIT killah|     Los del Espacio|   2023-06-01|     Los del Espacio|          85|
| Fito Paez|           EADDA9223|   2023-05-30|El Amor Después d...|          59|
| Fito Paez|           EADDA9223|   2023-05-30|Dos Días en la Vi...|          64|
| Fito Paez|           EADDA9223|   2023-05-30|La Verónica - EAD...|          58|
| Fito Paez|           EADDA9223|   2023-05-30|Tráfico por Katma...|          57|
| Fito Paez|           EADDA9223|   2023-05-30|Pétalo de Sal - E...|          57|
| Fito Paez|           EADDA9223|   2023-05-30|Sasha, Sissi y el...|          59|
| Fito Paez|           EADDA92

In [166]:
# suma la cantidad de reproducciones por cancion
print ('cant registros dataframe',df_final.count())
df_final = spark.sql("select Artista, Album, Track, Fecha_release, sum(Cant_reprod_track) as Cant_reprod_track from temp group by Artista, Album, Track, Fecha_release")
print ('cant registros luego de sumar',df_final.count())
df_final.show()

cant registros dataframe 852
cant registros luego de sumar 426
+----------------+--------------------+--------------------+-------------+-----------------+
|         Artista|               Album|               Track|Fecha_release|Cant_reprod_track|
+----------------+--------------------+--------------------+-------------+-----------------+
|           Cazzu|Nena Trampa (Delu...|Los Hombres No Ll...|   2023-04-13|               84|
|     Connie Isla|          Isla Mater|                 Gps|   2023-04-13|               46|
|     Connie Isla|          Isla Mater|              No Ves|   2023-04-13|               38|
|     Delfina Dib|     Directo al Cora|             ME OYES|   2023-04-14|               90|
|       Mel Muñiz|La Santa Patrona ...|     Toda esa bronca|   2023-03-14|               36|
|      Ñengo Flow|       Gato de Noche|       Gato de Noche|   2022-12-22|              176|
|    Foo Fighters|     But Here We Are|           Beyond Me|   2023-06-02|              134|
|      

In [167]:
# eliminacion de duplicados (aunque en el anterior paso vemos que ya se eliminan los duplicados)
print ('cant registros antes de borrar duplicados',df_final.count())
df_final=df_final.distinct()
print ('cant registros luego de borrar duplicados',df_final.count())

cant registros antes de borrar duplicados 426
cant registros luego de borrar duplicados 426


In [169]:
from pyspark.sql.functions import col, to_date

# cast del tipo string a date en la columna fecha_release
df_final = df_final.withColumn("Fecha_release", to_date(col("Fecha_release"), "yyyy-MM-dd"))
df_final.printSchema()
df_final.show()

root
 |-- Artista: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Track: string (nullable = true)
 |-- Fecha_release: date (nullable = true)
 |-- Cant_reprod_track: long (nullable = true)

+----------------+--------------------+--------------------+-------------+-----------------+
|         Artista|               Album|               Track|Fecha_release|Cant_reprod_track|
+----------------+--------------------+--------------------+-------------+-----------------+
|           Cazzu|Nena Trampa (Delu...|Los Hombres No Ll...|   2023-04-13|               84|
|     Connie Isla|          Isla Mater|                 Gps|   2023-04-13|               46|
|     Connie Isla|          Isla Mater|              No Ves|   2023-04-13|               38|
|     Delfina Dib|     Directo al Cora|             ME OYES|   2023-04-14|               90|
|       Mel Muñiz|La Santa Patrona ...|     Toda esa bronca|   2023-03-14|               36|
|      Ñengo Flow|       Gato de Noche|       G

In [170]:
# castea del tipo long a integer para la columna cant_reprod_track 
df_final = df_final.withColumn("Cant_reprod_track",col("Cant_reprod_track").cast("int"))
df_final.printSchema()
df_final.show()

root
 |-- Artista: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Track: string (nullable = true)
 |-- Fecha_release: date (nullable = true)
 |-- Cant_reprod_track: integer (nullable = true)

+----------------+--------------------+--------------------+-------------+-----------------+
|         Artista|               Album|               Track|Fecha_release|Cant_reprod_track|
+----------------+--------------------+--------------------+-------------+-----------------+
|           Cazzu|Nena Trampa (Delu...|Los Hombres No Ll...|   2023-04-13|               84|
|     Connie Isla|          Isla Mater|                 Gps|   2023-04-13|               46|
|     Connie Isla|          Isla Mater|              No Ves|   2023-04-13|               38|
|     Delfina Dib|     Directo al Cora|             ME OYES|   2023-04-14|               90|
|       Mel Muñiz|La Santa Patrona ...|     Toda esa bronca|   2023-03-14|               36|
|      Ñengo Flow|       Gato de Noche|     

ETAPA DE CARGA - Creación de tabla en Redshift y carga de datos del dataframe

In [192]:
connection = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD'],
)

In [193]:
# creacion de tabla en redshift  

cursor = connection.cursor()
cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.new_releases_aruy_spotify (
    artista VARCHAR(30) distkey,
    album VARCHAR(30),
    fecha_release DATE,
    track VARCHAR(30),
    cant_reprod_track INT
) sortkey(fecha_release);
""")
connection.commit()

cursor.close()
print("Table created!")

Table created!


In [184]:
#carga el dataframe en la tabla de redshift
df_final.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.new_releases_aruy_spotify") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [185]:
# chequeo con una consulta si los datos estan en la tabla creada en redshift
query = f"select * from {env['AWS_REDSHIFT_SCHEMA']}.new_releases_aruy_spotify"
data_redshift = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"({query}) as tmp_table") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [186]:
data_redshift.printSchema()
data_redshift.show()

root
 |-- artista: string (nullable = true)
 |-- album: string (nullable = true)
 |-- track: string (nullable = true)
 |-- fecha_release: date (nullable = true)
 |-- cant_reprod_track: integer (nullable = true)

+----------------+--------------------+--------------------+-------------+-----------------+
|         artista|               album|               track|fecha_release|cant_reprod_track|
+----------------+--------------------+--------------------+-------------+-----------------+
|           Cazzu|Nena Trampa (Delu...|Los Hombres No Ll...|   2023-04-13|               84|
|     Connie Isla|          Isla Mater|                 Gps|   2023-04-13|               46|
|     Connie Isla|          Isla Mater|              No Ves|   2023-04-13|               38|
|     Delfina Dib|     Directo al Cora|             ME OYES|   2023-04-14|               90|
|       Mel Muñiz|La Santa Patrona ...|     Toda esa bronca|   2023-03-14|               36|
|      Ñengo Flow|       Gato de Noche|     

In [190]:
connection.close()